In [ ]:
! pip install sentencepiece
!pip install transformers
!pip install jsonlines
!pip install wandb
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 6.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 3.9 MB/s 
     |████████████████████████████████| 6.6 MB 53.3 MB/s 
     |████████████████████████████████| 596 kB 51.1 MB/s 
     |████████████████████████████████| 86 kB 3.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 7.4 MB/s 
     |████████████████████████████████| 181 kB 49.7 MB/s 
     |████████████████████████████████| 145 kB 66.5 MB

In [ ]:
import numpy as np
import pandas as pd
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import transformers
from transformers import BertTokenizer, BertForSequenceClassification 
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import RobertaTokenizer, RobertaModel

from transformers import AdamW, Adafactor

import jsonlines

from sklearn.metrics.pairwise import cosine_similarity
import torch



import transformers
from transformers import RobertaTokenizer, AdamW, get_linear_schedule_with_warmup, RobertaModel, RobertaForSequenceClassification
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, TensorDataset, DataLoader
import time
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from transformers import get_linear_schedule_with_warmup

import warnings
warnings.filterwarnings("ignore")


print(torch.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

1.11.0+cu113


device(type='cuda', index=0)

In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
RANDOM_SEED = 28

In [ ]:
set_random_seed(RANDOM_SEED)

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
model_name = 'sberbank-ai/ruRoberta-large'

In [ ]:
#model_name = 'bert-base-multilingual-cased'

In [ ]:
class Dataset_TERRa(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_dict = {'not_entailment': 0, 'entailment': 1}
        
    def __getitem__(self, index):
        premise = self.data.iloc[index]['premise']
        hypothesis = self.data.iloc[index]['hypothesis']
        label = self.data.iloc[index]['label']
        pair_token_ids_padded = torch.zeros(self.max_len, dtype=torch.long)
        attention_mask_ids_padded = torch.zeros(self.max_len, dtype=torch.long)
        premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
        hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
        pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
        if len(pair_token_ids) < self.max_len:
            pair_token_ids_padded[:len(pair_token_ids)] = torch.tensor(pair_token_ids)
        else:
            pair_token_ids_padded = torch.tensor(pair_token_ids[:self.max_len])
        
        premise_len = len(premise_id)
        hypothesis_len = len(hypothesis_id)
        attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))
        if len(attention_mask_ids) < self.max_len:
            attention_mask_ids_padded[:len(attention_mask_ids)] = attention_mask_ids
        else:
            attention_mask_ids_padded = attention_mask_ids[:self.max_len]

        return {
            'input_ids': pair_token_ids_padded,
            'attention_mask': attention_mask_ids_padded,
            'labels': torch.tensor(self.label_dict[label], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [ ]:
!wget https://www.dropbox.com/s/6gq82kitvbykpoy/train.jsonl?dl=0 -O train.jsonl
!wget https://www.dropbox.com/s/su1urqud44t6yh5/val.jsonl?dl=0 -O val.jsonl
!wget https://www.dropbox.com/s/9e4894vphpwu88w/test.jsonl?dl=0 -O test.jsonl

--2022-05-26 23:00:47--  https://www.dropbox.com/s/6gq82kitvbykpoy/train.jsonl?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6gq82kitvbykpoy/train.jsonl [following]
--2022-05-26 23:00:47--  https://www.dropbox.com/s/raw/6gq82kitvbykpoy/train.jsonl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7473debb40f91dec67d39bad69.dl.dropboxusercontent.com/cd/0/inline/BmALLLiJgBJsWo1MlOfcyndF-hcIguPwS4unjsi2Vw-NZ59ZdjOq3b2TDyI233KtFxBJWg3GdKSG5P3zb0OjeTdd66gRcezNGXLbeWFzlpAGa8CbXutfe2PvyoKelbOKxzhDJC9E-mBw80MoEeYklz1WpSNXL__RN3Pfs5HhbjK41A/file# [following]
--2022-05-26 23:00:47--  https://uc7473debb40f91dec67d39bad69.dl.dropboxusercontent.com/cd/0/inline/BmALLLiJgBJsWo1MlOfcyndF-hcIguPwS4unjsi2Vw-NZ59ZdjOq3b2TDyI233KtF

In [ ]:
def get_data(filename):
    data = []
    with jsonlines.open(f'/content/{filename}.jsonl') as reader:
        for obj in reader:
            data.append(obj)
    return pd.DataFrame(data)

In [ ]:
train_df = get_data('train')
val_df = get_data('val')
test_df = get_data('test')

In [ ]:
train_df.shape, val_df.shape, test_df.shape

((2616, 4), (307, 4), (3198, 3))

In [ ]:
train_df['label'].unique()

array(['entailment', 'not_entailment'], dtype=object)

In [ ]:
NUM_CLASSES = len(train_df['label'].unique())
NUM_CLASSES

2

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

In [ ]:
attack2_test = pd.read_csv('/content/attack2_test.csv')

In [ ]:
train_dataset = Dataset_TERRa(train_df, tokenizer, 512)
val_dataset = Dataset_TERRa(val_df, tokenizer, 512)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    model_name, 
    num_labels = NUM_CLASSES, # The number of output labels
)

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Some weights of the model checkpoint at sberbank-ai/ruRoberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruRoberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.o

In [ ]:
to_train = [
             'classifier.out_proj.weight', 'classifier.out_proj.bias', 
             "classifier.dense.weight", "classifier.dense.bias"
]

In [ ]:
for name,param in model.named_parameters():
    if name not in to_train:
        param.requires_grad = False
    else:
      print(name)

classifier.dense.weight
classifier.dense.bias
classifier.out_proj.weight
classifier.out_proj.bias


In [ ]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

## Train

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_ratio=0.06,                # number of warmup steps for learning rate scheduler
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=3e-5,
    evaluation_strategy="epoch",
)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics,    # metrics to evaluate
    data_collator=data_collator
)

trainer.train()

***** Running training *****
  Num examples = 2616
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 820
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.704000,0.701890,0.488599,0.642369,0.493007,0.921569
2,0.678700,0.687880,0.563518,0.521429,0.574803,0.477124
3,0.671900,0.695715,0.495114,0.637002,0.496350,0.888889
4,0.675800,0.692476,0.504886,0.631068,0.501931,0.849673
5,0.702600,0.688932,0.576547,0.606061,0.564972,0.653595


***** Running Evaluation *****
  Num examples = 307
  Batch size = 16
***** Running Evaluation *****
  Num examples = 307
  Batch size = 16
***** Running Evaluation *****
  Num examples = 307
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 307
  Batch size = 16
***** Running Evaluation *****
  Num examples = 307
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=820, training_loss=0.6954695573667201, metrics={'train_runtime': 1783.6597, 'train_samples_per_second': 7.333, 'train_steps_per_second': 0.46, 'total_flos': 1.218966223233024e+16, 'train_loss': 0.6954695573667201, 'epoch': 5.0})

In [ ]:
outputs = trainer.predict(val_dataset)
y_pred = outputs.predictions.argmax(1)

***** Running Prediction *****
  Num examples = 307
  Batch size = 16


In [ ]:
val_df['ans'] = y_pred


In [ ]:
val_df['label_int'] = val_df['label'].map({'not_entailment': 0, 'entailment':1})

In [ ]:
val_df.to_csv('ruRoberta_full_clear_answers.csv', index=False)

In [ ]:
def make_attack(name, y_pred):
  attck1 = pd.read_csv(f'/content/{name}.csv')
  attck1_df = Dataset_TERRa(attck1, tokenizer, 512)
  outputs1 = trainer.predict(attck1_df)
  y_pred1 = outputs1.predictions.argmax(1)
  attck1['ans'] = y_pred1
  attck1['label_int'] = attck1['label'].map({'not_entailment': 0, 'entailment':1})
  attck1['clear_ans'] = y_pred
  attck1[attck1['ans'] != attck1['clear_ans']].to_csv(f'ruRoberta_{name}_mistakes.csv', index=False)
  attck1.to_csv(f'ruRoberta_{name}_answers.csv', index=False)

In [ ]:
make_attack('attack2_val', y_pred)

***** Running Prediction *****
  Num examples = 307
  Batch size = 16


In [ ]:
make_attack('attack3_1_val', y_pred)

***** Running Prediction *****
  Num examples = 307
  Batch size = 16


In [ ]:
make_attack('attack3_2_val', y_pred)

***** Running Prediction *****
  Num examples = 307
  Batch size = 16


In [ ]:
make_attack('attack4_val', y_pred)

***** Running Prediction *****
  Num examples = 307
  Batch size = 16
